In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/id-to-summary/id_to_summary_lines.pickle
/kaggle/input/5-million-song-lyrics-dataset/ds2.csv
/kaggle/input/wasabi-songs/wasabi_songs.csv
/kaggle/input/combined-partial/combined_second.csv
/kaggle/input/combined-partial/combined.csv
/kaggle/input/combined-partial/combined_first_tags.csv


In [2]:
import time
import csv
from io import BytesIO

In [3]:
id_to_summary = pd.read_pickle('/kaggle/input/id-to-summary/id_to_summary_lines.pickle')

In [4]:
len(id_to_summary)

53124

In [5]:
list_summaries_id = list(id_to_summary)

In [6]:
set_summaries_id = set(list(id_to_summary))

In [7]:
id_to_summary['5714dec325ac0d8aee3806e4']

["'cause you were always on the run",
 'from the darkness in your heart',
 'so you wear it on the outside',
 'from the poison in your lungs']

In [8]:
title_to_text = pd.read_csv('/kaggle/input/5-million-song-lyrics-dataset/ds2.csv', on_bad_lines='skip', usecols = ['title', 'lyrics', 'artist', 'tag'])  

In [9]:
title_to_text.sample()

,title,tag,artist,lyrics
5758066,Gonka,pop,LUV3MEMORE,[Текст песни «Gonka»]\n\n[Припев]\nМоя жизнь —...


In [10]:
title_with_text_set = set()
for title in title_to_text.iloc[:,0]:
    title_with_text_set.add(title)

In [11]:
id_to_title = pd.read_csv('/kaggle/input/wasabi-songs/wasabi_songs.csv', on_bad_lines='skip', sep ='\t', usecols = ['_id', 'title', 'artist'])
id_to_title['_id'] = id_to_title['_id'].apply(lambda s: s[9:-1])

In [12]:
id_to_title.sample()

,_id,artist,title
1294511,5714dedd25ac0d8aee4bc50b,My Autumn Empire,Every Now And Then I Fall Apart


In [13]:
i = 0
for object_id in id_to_title.iloc[:,0]:
    i+=1
    print(object_id)
    if i > 10:
        break

5714dec325ac0d8aee3804e7
5714dec325ac0d8aee3804e8
5714dec325ac0d8aee3804e9
5714dec325ac0d8aee3804ea
5714dec325ac0d8aee3804eb
5714dec325ac0d8aee3804ec
5714dec325ac0d8aee3804ed
5714dec325ac0d8aee3804ee
5714dec325ac0d8aee3804ef
5714dec325ac0d8aee3804f0
5714dec325ac0d8aee3804f1


In [14]:
i = 0
check_set = set()
for object_id in id_to_title.iloc[:,0]:
    #if object_id[9:-1] in set_summaries_id and object_id[:9] == 'ObjectId(' and not object_id[9:-1] in check_set:
    if object_id in set_summaries_id and not object_id in check_set:
        check_set.add(object_id)
        i+=1
print('amount of ids with summaries found: ', i)
# 53133 all

amount of ids with summaries found:  53124


In [15]:
i = 0
for _id in check_set:
    print('_id ', _id)
    i += 1
    if i > 10:
        break

_id  5714decb25ac0d8aee3e61ed
_id  5714ded825ac0d8aee485c7b
_id  5714dedb25ac0d8aee4a3a8f
_id  5714dee925ac0d8aee558726
_id  5714ded725ac0d8aee474bf4
_id  5714dee125ac0d8aee4f6835
_id  5714dec725ac0d8aee3b435d
_id  5714dee925ac0d8aee55913b
_id  5714dec725ac0d8aee3b633b
_id  5714decd25ac0d8aee400f8f
_id  5714ded525ac0d8aee46340c


In [16]:
list(id_to_title)

['_id', 'artist', 'title']

In [17]:
'''
count_match = 0
check_set = set()
for i in range(len(id_to_title)):
    _id = id_to_title.iloc[i, 0]
    title = id_to_title.iloc[i, 1]
    #if (_id[9:-1] in set_summaries_id) and (_id[:9] == 'ObjectId(') and (not _id[9:-1] in check_set) and (title in title_with_text_set):
    if (_id in set_summaries_id) and (not _id in check_set) and (title in title_with_text_set):
        check_set.add(_id)
        count_match += 1
print('matched summary to text: ', count_match)

# 23107
'''

"\ncount_match = 0\ncheck_set = set()\nfor i in range(len(id_to_title)):\n    _id = id_to_title.iloc[i, 0]\n    title = id_to_title.iloc[i, 1]\n    #if (_id[9:-1] in set_summaries_id) and (_id[:9] == 'ObjectId(') and (not _id[9:-1] in check_set) and (title in title_with_text_set):\n    if (_id in set_summaries_id) and (not _id in check_set) and (title in title_with_text_set):\n        check_set.add(_id)\n        count_match += 1\nprint('matched summary to text: ', count_match)\n\n# 23107\n"

In [18]:
#combined_df = pd.DataFrame(columns=['title', 'tag', 'unredacted_text', 'clean_text', 'id', 'artist', 'unredacted_summary', 'clean_summary'])
combined_df = pd.read_csv('/kaggle/input/combined-partial/combined_first_tags.csv')

In [19]:
#combined_df.iloc[-1]['id']

In [20]:
#list_summaries_id.index(combined_df.iloc[-1]['id'])

In [21]:
# ''' '''
start = time.time()
if len(combined_df) > 0:
    count = len(combined_df)
    count_all = list_summaries_id.index(combined_df.iloc[-1]['id'])
    start_from = (list_summaries_id.index(combined_df.iloc[-1]['id']) + 1)
else:
    count = 0
    count_all = 0
    start_from = 0

with open('combined_second_tags.csv', 'w', newline='') as csvfile:
    fieldnames = ['title', 'tag', 'unredacted_text', 'clean_text', 'id', 'artist', 'unredacted_summary', 'clean_summary']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for _id in list_summaries_id[start_from:]:
        count_all += 1

        if count_all % 1000 == 0:
            #combined_df = pd.read_csv('/kaggle/working/combined.csv')
            #combined_df.to_csv('combined_df_coma.csv', sep=',')
            print('count_all ', count_all)
            print('count ', count)
            end = time.time()
            print(end - start)

        song_with_id = id_to_title.loc[id_to_title['_id'] == _id]
        title = song_with_id['title'].iloc[0]
        artist = song_with_id['artist'].iloc[0]

        #candidate = title_to_text.loc[(title_to_text['title'] == title) & (title_to_text['artist'] == artist)]
        #candidate = title_to_text.loc[(title_to_text['title'] == title)].loc[(title_to_text['artist'] == artist)]
        candidate = title_to_text.loc[(title_to_text['artist'] == artist)].loc[(title_to_text['title'] == title)]

        if len(candidate) > 0:
            count += 1
            unredacted_text = candidate['lyrics'].iloc[0]
            tag = candidate['tag'].iloc[0]
            summary = id_to_summary[_id]
            
            writer.writerow({'title': title, 'tag': tag, 'unredacted_text': unredacted_text, 'clean_text': '', 'id': _id, 'artist': artist, 'unredacted_summary': summary , 'clean_summary': ''})

    print('count_all ', count_all)
    print('count ', count )
# '''

count_all  47000
count  31316
1059.0020163059235
count_all  48000
count  31958
2164.679279088974
count_all  49000
count  32613
3268.8733716011047
count_all  50000
count  33268
4376.101979494095
count_all  51000
count  33873
5482.046710014343
count_all  52000
count  34532
6585.010985612869
count_all  53000
count  35194
7689.110816717148
count_all  53123
count  35281


In [22]:
'''
start = time.time()
if len(combined_df) > 0:
    count = len(combined_df)
    count_all = ids_summary.index(combined_df.iloc[-1]['id'])
else:
    count = 0
    count_all = 0
    
with open('combined.csv', 'w', newline='') as csvfile:
    fieldnames = ['title', 'unredacted_text', 'clean_text', 'id', 'artist', 'unredacted_summary', 'clean_summary']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for index, row in id_to_title.iterrows():
        _id = row['_id']
        count_all += 1

        if count_all % 1000 == 0:
            #combined_df.to_csv('combined_df_coma.csv', sep=',')
            print('count_all ', count_all)
            print('count ', count)
            end = time.time()
            print(end - start)
            
        if _id in set_summaries_id:
            title = row['title']
            artist = row['artist']

            # candidate = title_to_text.loc[(title_to_text['title'] == title) & (title_to_text['artist'] == artist)]
            candidate = title_to_text.loc[(title_to_text['title'] == title)].loc[(title_to_text['artist'] == artist)]
            
            if len(candidate) > 0:
                count += 1
                unredacted_text = candidate['lyrics'].iloc[0]
                summary = id_to_summary[_id]

                writer.writerow({'title': title, 'unredacted_text': unredacted_text, 'clean_text': '', 'id': _id, 'artist': artist, 'unredacted_summary': summary , 'clean_summary': ''})

    print('count_all ', count_all)
    print('count ', count )
'''

"\nstart = time.time()\nif len(combined_df) > 0:\n    count = len(combined_df)\n    count_all = ids_summary.index(combined_df.iloc[-1]['id'])\nelse:\n    count = 0\n    count_all = 0\n    \nwith open('combined.csv', 'w', newline='') as csvfile:\n    fieldnames = ['title', 'unredacted_text', 'clean_text', 'id', 'artist', 'unredacted_summary', 'clean_summary']\n    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)\n    writer.writeheader()\n    \n    for index, row in id_to_title.iterrows():\n        _id = row['_id']\n        count_all += 1\n\n        if count_all % 1000 == 0:\n            #combined_df.to_csv('combined_df_coma.csv', sep=',')\n            print('count_all ', count_all)\n            print('count ', count)\n            end = time.time()\n            print(end - start)\n            \n        if _id in set_summaries_id:\n            title = row['title']\n            artist = row['artist']\n\n            # candidate = title_to_text.loc[(title_to_text['title'] == title) &

In [23]:
#len(id_to_title)

In [24]:
#combined_df = pd.read_csv('/kaggle/working/combined.csv')
#combined_df

In [25]:
#combined_df.to_csv('combined_df.csv', sep=',')

In [26]:
#!ls /kaggle/working

In [27]:
#combined_df.iloc[-1]['id']

In [28]:
#list_summaries_id.index(combined_df.iloc[-1]['id'])

In [29]:
#combined_df.iloc[-1]['id']

In [30]:
#list_summaries_id[list_summaries_id.index(combined_df.iloc[-1]['id'])]

In [31]:
#test_read_pd = pd.read_csv('/kaggle/input/combined-df-coma-partial/combined_df_coma.csv', sep = ',')  

In [32]:
#test_read_pd.sample(10)

In [33]:
#test_read_pd['unredacted_text'].nunique()

In [34]:
#test_read_pd['unredacted_text'].count()

In [35]:
#test_read_pd['unredacted_text'].size